# IMDB 영화평 감성분석(이진 분류법)

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/labeledTrainData.tsv',sep='\t',quoting=3)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


- 전처리

In [3]:
# <br / > 태그는 공백으로 변환
df.review = df.review.str.replace('<br/>',' ')

In [4]:
df.review = df.review.str.replace('br',' ')

In [5]:
# 구둣점, 숫자 제거 -- > 영어 이외의 문자는 공백으로 변환
df.review = df.review.str.replace('[^A-Za-z]',' ')

<ipython-input-5-e5ec7f415b7f>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.review = df.review.str.replace('[^A-Za-z]',' ')


In [6]:
df['review'][0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay           Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him           The actual feature film bit when it finally starts

train/test dataset 으로 분리

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
    df['review'],df['sentiment'] , stratify= df['sentiment'] , random_state=2021
)
y_train.value_counts(),y_test.value_counts()

(0    9375
 1    9375
 Name: sentiment, dtype: int64,
 0    3125
 1    3125
 Name: sentiment, dtype: int64)

- countVectorizer로 변환

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english',ngram_range=(1,2))
cvect.fit(X_train)

CountVectorizer(ngram_range=(1, 2), stop_words='english')

이런 방법으로 하면 안됨

In [9]:
# cvect.fit_transform(X_train).shape, cvect.fit_transform(X_test).shape

이와 같은 방법을 사용해야 함

In [10]:
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape , X_test_cv.shape

((18750, 1383400), (6250, 1383400))

- LogisticRegression

In [11]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(max_iter=300)
%time lrc.fit(X_train_cv,y_train)


Wall time: 29.9 s


LogisticRegression(max_iter=300)

In [12]:
lrc.score(X_test_cv,y_test)

0.88672

- TFidfVectorizer로 변환/학습/평가

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
tvect.fit(X_train)
X_train_tv = tvect.transform(X_train)
X_test_tv = tvect.transform(X_test)
X_train_tv.shape, X_test_tv.shape

((18750, 1383400), (6250, 1383400))

In [14]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(max_iter=300)
%time lrc.fit(X_train_tv,y_train)
lrc.score(X_test_tv,y_test)

Wall time: 11.8 s


0.87888

In [15]:
y_train[0:10],pd.Series(lrc.predict(X_test_cv)[0:10])

(3472     1
 21508    0
 4527     1
 23575    1
 14763    1
 19502    1
 18442    1
 16562    0
 1062     0
 15513    0
 Name: sentiment, dtype: int64,
 0    1
 1    1
 2    0
 3    1
 4    0
 5    1
 6    0
 7    0
 8    0
 9    0
 dtype: int64)

In [16]:
import joblib

In [17]:
joblib.dump(tvect, 'model/imab_tvect.pkl')
joblib.dump(lrc,'model/imab_lrc.pkl')


['model/imab_lrc.pkl']

In [18]:
new_tvect = joblib.load('model/imab_tvect.pkl')
new_lrc = joblib.load('model/imab_lrc.pkl')

In [19]:
new_X_test_tv = new_tvect.transform(X_test)
new_lrc.score(new_X_test_tv,y_test)

0.87888

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC # svc
svc = SVC()
from sklearn.tree import DecisionTreeClassifier # Dt
dt = DecisionTreeClassifier()
from sklearn.ensemble import RandomForestClassifier # RF
rf = RandomForestClassifier()
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
# for index in ['svc','dt','rf']:
#     for arg in ['cv','tv']:
#             for i in range (1,2):
#                 globals()[f'{index}'].fit(
#                     globals()[f'X_test_{arg}_{i}'],
#                     globals()[f'y_test_{arg}_{i}']
#                 )
#                 globals()[f'{index}'].score(
#                     globals()[f'X_test_{arg}_{i}'],
#                     globals()[f'y_test_{arg}_{i}']
#                 )         